In [1]:
import tensorflow as tf
import numpy as np
import json
import copy

In [2]:
def loadMaps():
    with open('maps1.json', 'r') as f:
        a = json.load(f)
    return copy.deepcopy(a)

In [3]:
maps = loadMaps()
# SHAPE OF THE DATA
print(np.array(maps["maps"]).shape)
# SHAPE OF THE LABELS
print(np.array(maps["moves"]).shape)

(576225, 400)
(576225,)


In [4]:
from sklearn.preprocessing import OneHotEncoder

npSolution = np.array(maps["moves"]).reshape(len(maps["moves"]), 1)
onehot_encoder = OneHotEncoder(sparse=False)
encoded = onehot_encoder.fit_transform(npSolution)

/home/milkylatte/miniconda3/envs/tf/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(maps["maps"]), encoded, test_size=0.3, random_state = 101)

In [ ]:
train_X = X_train.reshape(-1, 20, 20, 1)
test_X = X_test.reshape(-1, 20, 20, 1)

In [ ]:
train_X.shape, test_X.shape


((403357, 20, 20, 1), (172868, 20, 20, 1))

In [ ]:
training_iters = 250
learning_rate = 0.0001
batch_size = 512

In [ ]:
n_input = 20
n_classes = 4

In [ ]:
x = tf.placeholder("float", [None, 20, 20, 1])
y = tf.placeholder("float", [None, n_classes])

In [ ]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [ ]:
weights = {
    'wc1': tf.get_variable('W0', shape=(3,3,1,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc2': tf.get_variable('W1', shape=(3,3,32,64), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc3': tf.get_variable('W2', shape=(3,3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'wd1': tf.get_variable('W3', shape=(3*3*128,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('W6', shape=(128,n_classes), initializer=tf.contrib.layers.xavier_initializer()), 
}
biases = {
    'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B4', shape=(4), initializer=tf.contrib.layers.xavier_initializer()),
}

In [ ]:
def conv_net(x, weights, biases):  
    # here we call the conv2d function we had defined above and pass the input image x, weights wc1 and bias bc1.
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 14*14 matrix.
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    # here we call the conv2d function we had defined above and pass the input image x, weights wc2 and bias bc2.
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 7*7 matrix.
    conv2 = maxpool2d(conv2, k=2)

    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 4*4.
    conv3 = maxpool2d(conv3, k=2)


    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [ ]:
pred = conv_net(x, weights, biases)

hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(pred, keep_prob=hold_prob)

with tf.name_scope('cross_entropy'):
    with tf.name_scope('total'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
tf.summary.scalar('cross_entropy', cost)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    with tf.name_scope('accuracy'):
        #calculate accuracy across all the given images and average them out. 
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

In [ ]:
init = tf.global_variables_initializer()

merged = tf.summary.merge_all()



In [ ]:
with tf.Session() as sess:
    sess.run(init) 
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    train_writer = tf.summary.FileWriter('./train', sess.graph)
    test_writer = tf.summary.FileWriter('./test', sess.graph)
    for i in range(training_iters):
        for batch in range(len(train_X)//batch_size):
            batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
            batch_y = y_train[batch*batch_size:min((batch+1)*batch_size,len(y_train))]    
            # Run optimization op (backprop).
                # Calculate batch loss and accuracy
            opt = sess.run(optimizer, feed_dict={x: batch_x,
                                                              y: batch_y,
                                                               hold_prob: 0.1})
            summary,loss, acc = sess.run([merged,cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y, hold_prob: 0.1})
            train_writer.add_summary(summary, i)
        if i % 10 == 0:
            print("Iter " + str(i) + ", Loss= " + \
                          "{:.6f}".format(loss) + ", Training Accuracy= " + \
                          "{:.5f}".format(acc))
            summary, test_acc,valid_loss = sess.run([merged, accuracy,cost], feed_dict={x: test_X[:10000],y : y_test[:10000]})
            test_writer.add_summary(summary, i)
#             train_loss.append(loss)
#             test_loss.append(valid_loss)
#             train_accuracy.append(acc)
#             test_accuracy.append(test_acc)
            print("Testing Accuracy:","{:.5f}".format(test_acc))
    
    print("Optimization Finished!")

    # Calculate accuracy for all 10000 mnist test images

    test_writer.close()
    train_writer.close()

Iter 0, Loss= 1.385832, Training Accuracy= 0.26758
Testing Accuracy: 0.25890
Iter 10, Loss= 1.333596, Training Accuracy= 0.34961
Testing Accuracy: 0.35190
Iter 20, Loss= 1.193702, Training Accuracy= 0.44141
Testing Accuracy: 0.45630
Iter 30, Loss= 0.982383, Training Accuracy= 0.58398
Testing Accuracy: 0.57440
Iter 40, Loss= 0.795351, Training Accuracy= 0.66406
Testing Accuracy: 0.68760
Iter 50, Loss= 0.602229, Training Accuracy= 0.77734
Testing Accuracy: 0.76640
Iter 60, Loss= 0.486765, Training Accuracy= 0.83398
Testing Accuracy: 0.81140
Iter 70, Loss= 0.439233, Training Accuracy= 0.85938
Testing Accuracy: 0.83820
Iter 80, Loss= 0.402246, Training Accuracy= 0.86133
Testing Accuracy: 0.85610
